In [1]:
import cv2
import time
from collections import deque
import argparse
import os

from glob import glob

In [2]:
def txt2vidoname_ses(data_dir, txtfile):
    fn=os.path.basename(txtfile).split(".")[0]+".mp4"
    video_fn=data_dir+fn 
    with open(txtfile, "r") as f:
        lines=f.readlines()
        
    action=lines[1].strip()
    line=lines[2].strip()

    ses=eval(line)
    return video_fn, ses, action

In [3]:
def video2clips(data_dir, video_fn, ses, action_name):
    """
    ses: [start,end, start2,end2, ...]
    """
    if not os.path.exists(video_fn):
        video_fn=video_fn.replace("mp4", "mov")
    if not os.path.exists(video_fn):
        video_fn=video_fn.replace("mov", "MOV")

    print('-'*10, "\nopening video file=", video_fn, " len(ses)=", len(ses))
    video=cv2.VideoCapture(video_fn)
    imgs=[]
    i=0

    clip_id=0
    s,e=ses[clip_id*2], ses[clip_id*2+1]
    while(video.isOpened()):
        ret, frame = video.read()
        if ret==False: break
        
        if i>=s and i<=e:
            imgs.append(frame)
        elif i>e: 
            #save imgs as clip video
            fn=os.path.basename(video_fn).split(".")[0]
            clip_fn=data_dir+f"clips_{action_name}\\"+fn.split(".")[0]+"_"+str(clip_id)+".mp4"
            print("saving clip_fn=", clip_fn , " se: ", s,e, " len(imgs): ", len(imgs) )
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(clip_fn,fourcc, 30.0, (imgs[0].shape[1],imgs[0].shape[0]))
            for img in imgs:
                out.write(img)
            out.release()


            clip_id+=1
            if clip_id < ( len(ses)/2 ):
                s,e=ses[clip_id*2], ses[clip_id*2+1]
                imgs=[]  
            else:
                break

        i+=1
        

    video.release() 

In [4]:
data_dir="C:\\Users\\noush\\Downloads\\Eating\\"
meta_dir=data_dir+"txts_noteating\\"
files=glob(meta_dir+"*.txt")
files

['C:\\Users\\noush\\Downloads\\Eating\\txts_noteating\\Dain Eating 1.txt',
 'C:\\Users\\noush\\Downloads\\Eating\\txts_noteating\\Dain Eating 2.txt',
 'C:\\Users\\noush\\Downloads\\Eating\\txts_noteating\\Eating Pasta.txt',
 'C:\\Users\\noush\\Downloads\\Eating\\txts_noteating\\eating_sample.txt']

In [5]:
# video_fn, ses, action=txt2vidoname_ses(data_dir, files[0])
# video2clips(data_dir, video_fn, ses, action)

In [6]:
#get filename
file_id=0
file_id=1

for file_id in range(len(files)):
    video_fn, ses, action=txt2vidoname_ses(data_dir, files[file_id])
    video2clips(data_dir, video_fn, ses, action) 

---------- 
opening video file= C:\Users\noush\Downloads\Eating\Dain Eating 1.mp4  len(ses)= 18
saving clip_fn= C:\Users\noush\Downloads\Eating\clips_noteating\Dain Eating 1_0.mp4  se:  8 205  len(imgs):  198
